In [1]:
from analysis.tax import AutoCGT

cgt = AutoCGT()
cgt.fy()


Capital gains for 	FY2019-2020
Total capital gains:	 $ 11949.16
        


,CapitalGain
Ticker,
AVH,428.623957
BBOZ,177.800000
BBUS,4943.394625
EHE,73.225000
IFN,77.714914
NAN,942.960362
PTX,180.509891
QAN,146.000000
TWE,59.930000
